In [13]:
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
from sklearn import linear_model
import requests
from sklearn import metrics
data_dir = './vaccine_data/hhs_data'

### Data Splits

In [25]:
train_years = ['2012','2013','2014']
dev_years = ['2015']
test_years = ['2016','2017']

train_download_dir = ('train',train_years)
dev_download_dir = ('dev',dev_years)
test_download_dir = ('test',test_years)
data_split_ls = [train_download_dir,dev_download_dir,test_download_dir]

### URL To get data from https://fluvaccineapi.hhs.gov

In [35]:
get_state_id_url = "https://fluvaccineapi.hhs.gov/api/v2/ids/{}/states.json"
get_national_trends_url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/" +\
"{}/national.json?ethnicity=T&medicare_status=A"

get_state_trends_url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/"+\
"{}/states/{}.json?ethnicity=T&medicare_status=A"


## Code to download data

In [39]:
os.makedirs(data_dir,exist_ok = True)
for data_split in data_split_ls:
    split,years = data_split
    curr_dir = os.path.join(data_dir,split)
    os.makedirs(curr_dir,exist_ok = True)
    for year in years:
        curr_year_dir = os.path.join(curr_dir,year)
        os.makedirs(curr_year_dir,exist_ok = True)
        
        response = requests.get(get_national_trends_url.format(year))
        national_trends_json = response.json()
        with open(curr_year_dir+'/national.json', 'w') as outfile:
                    json.dump(national_trends_json, outfile)
        response = requests.get(get_state_id_url.format(year))
        
        states = response.json()
        for state in states:
            response = requests.get(get_state_trends_url.format(year,state))
            state_trends_json = response.json()
            with open(curr_year_dir+'/{}.json'.format(state), 'w') as outfile:
                json.dump(state_trends_json, outfile)
            